# KNIT7
# Import ELK Snapshots onto a Fabric slice

### Prerequisite
- A running fabric slice (For example: [Create single node backup Slice](./knit7_create_single_node_backup.ipynb))
- An ELK snapshot tar file to import

#### Set slice information and import ELK service tool
*Set slice_name and node_name below before running*

In [ ]:
slice_name='backup-testing'
node_name='data-backup'

from mflib.data_transfer import ElkImporter
elk_tools = ElkImporter(slice_name=slice_name, node_name=node_name)

---
---
# Instructions to start ELK and import your snapshot data

## Set up the Docker-Compose App
This command does the following:
- Installs Docker and Docker-Compose (if they aren't already)
- Sets up NAT64 server
- Clones Docker-Compose git repository (https://github.com/fabric-testbed/mf-data-import-containers.git)

In [ ]:
elk_tools.setup_docker_app(node_name)

---
## Upload snapshot tar file to your node with 1 of the methods below

#### Option 1: Upload to node from local device with SCP

##### Prerequisite
- For this step, you need your local ssh environment configured ([Configure Environment](./configure.ipynb))

##### Example SCP (Secure Copy) command to download snapshot file from VM

scp -F ~/.ssh/fabric/ssh_config -i ~/.ssh/fabric/slice_key ubuntu@\[2620:0:c80:1001:f816:3eff:fe81:80cc]:/home/mfuser/services/elk/files/snapshots/snapshot.tar .

##### Fill in the variables below then run to generate your custom SCP command

In [ ]:
# Path of directory containing your snapshot file
directory_path = "/your/directory/"

# Snapshot file name
snapshot_file_name = "snapshot_test.tar"

print(elk_tools.generate_scp_upload_command(snapshot_file_name, directory_path))

##### More info
Fabric guide for transferring data directly from VM to local machine:
https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/

### Option 2: Upload from Jupyter Hub server

##### Use this method if your tar file is in the snapshots directory on this JH server

In [ ]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_file_name = "snapshot_test.tar"

# Uploads TAR file to your node
elk_tools.upload_snapshot(snapshot_file_name)

---
#### Once the file is on the meas_node, import it into the docker containers shared volume

In [ ]:
elk_tools.import_snapshot(snapshot_file_name)

---
#### Start the docker compose app

In [ ]:
elk_tools.start_docker()

#### Register the repository

In [ ]:
# Choose a name for the repository your snapshot will be imported into or keep as default
repository_name = "imported_repository"

elk_tools.register_repository(repository_name)

#### View repository after registration to get the snapshot name

In [ ]:
elk_tools.view_repository(repository_name)

#### Set the snapshot_name variable from the above output (snapshot_name and snapshot_file_name are different)

##### Example:
this_is_the_snapshot_name SUCCESS 1685748801 23:33:21 1685748801 23:33:21 400ms 10 10 0 10

In [ ]:
# SET THIS VARIABLE
snapshot_name = "snapshot_test"

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [ ]:
elk_tools.restore_snapshot(repository_name, snapshot_name)

### Your data is now imported on the node.

----
### View the index list now on ELK
You should see all of your indices with "restored-" appended to the front of their names.

In [ ]:
elk_tools.view_indices()

----
### View data from Kibana

#### On your local terminal create a tunnel with:

In [ ]:
node_ip = elk_tools.node.get_management_ip()
print(f'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}')

#### Then go to http://localhost:10030
---

### To reuse slice with new data (if you are done then just delete the slice)
#### Take down container and remove data

In [ ]:
elk_tools.stop_docker()
elk_tools.remove_data()

#### Rerun steps starting from *Create Docker container and import your data*